In [ ]:
#Google Drive baglanti
from google.colab import drive
drive.mount('/content/gdrive') 
%cd gdrive/MyDrive/Derin Öğrenme/  
%ls

In [ ]:
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator  # veriçoğaltmayı bu fonk ile yapılır
from keras.models import Sequential  #bir model içinde ard arıdına gelen konvolüsyonlar , poolingler, full connectedları sıralı olarak birbirine bağlayan fonk 
from keras.layers import Conv2D, MaxPooling2D 
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization, LayerNormalization  #dense=fulllyconnected layer
from keras import backend as K
#keras üzerinden yappıcaz bu örnekte. veri 

#giris olarak verilecek görüntülerin boyutları
img_width, img_height = 224, 224    #girişleri kaça kaç eğitmek istersin

train_data_yolu= 'elma/train'            ## 'data/train'
validation_data_yolu= 'elma/test'        ##'data/validation'   # validasyonu kullanmışız test kullanmamaışız. tam tersi de olabilirdi.
train_ornek_sayisi=2100 
validation_ornek_sayisi=300

if K.image_data_format()== 'channels_first':  #resimlerin ilk kanalını almayı sağla. rgb katmanından ilk katmanı yani
  input_shape= (3, img_width, img_height)
else:
  input_shape= (img_width, img_height, 3)

  batch_size=10 # her seferinde eğitim için alınacak veri miktarı - kaçar kçar örnek alıcak? mesela 10 örneği 5er 5er mi alıp eğitsin yoksa 1er 1er mi
#train_ornek_sayisi, validation_onrke_sayisi ile batch_size tam bölünmeli   !!!!

#train işlemi için kullanılan verileri cogaltıyor. ImageDataGenerator ile ön işlem yapıyoruz
train_datalar=ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)
#zoomlayıp kırpabilisrsin, shear entropi işlemleridir, horizantal eksenine göre flip yaz (istersek vertical ile de flip yapabiliriz.), rescale ile de normalizasyon yapar 0-255 arasını 0-1 arasına çeker
#datalar = ImageDataGenerator(rotation_range=40, width_shift_range=0.2, heigth_shift_range=0.2,
#       shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')
# istersek rotation ile döndürme, shift ile kaydırma,  fillmode nearest ile de en yakınındaki piksele göre komple doldurma işlemi, 
#test işlemi için görüntüleri rescale boyutlandırma yapıyor
test_datalar =ImageDataGenerator(rescale=1. / 255)

train_generator=train_datalar.flow_from_directory(
    train_data_yolu,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')
#binary classification : class_mode='binary'   yazardık
# multiclass (coklu sınıf): class_mode='categorical' 

validation_generator= test_datalar.flow_from_directory(
    validation_data_yolu,
    target_size=(img_width,img_height),
    batch_size=batch_size,
    class_mode='categorical')
#binary classification : class_mode='binary'   yazardık
# multiclass (coklu sınıf): class_mode='categorical' 

Found 2100 images belonging to 3 classes.
Found 300 images belonging to 3 classes.


In [ ]:
import random

con_func = ['relu']   # ['relu', 'sigmoid'] # tanh denedim 33 verdi
traning_func = [ 'relu','tanh','sigmoid']  #
transfer_func = ['softmax','sigmoid']  #'sigmoid', 

for i in con_func:
  sayac = 1
  for j in traning_func:
    for m in transfer_func:

      epochs= random.randint(50, 150)    # öğrenme sayısı
      print(f"Conv Function: {i}, Traning Function: {j}, Transfer Function: {m}, Epochs: {epochs}")
      #Alexnet MODEL OLUŞTURMA

      model = Sequential()

      #Giris katmani ve Conv1
      #model.add(Conv2D(32,kernel_size=(5,5),padding='same',input_shape=input_shape, activation=i))  # same değil 1,1
      #model.add(BatchNormalization())
      #model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
      #model.add(Conv2D(64,kernel_size=(3,3),padding='same',activation=i))  # same değil 1,1
      #model.add(BatchNormalization())

      model.add(Conv2D(64,kernel_size=(3,3),padding='same')) 
      model.add(Conv2D(64,kernel_size=(3,3),padding='same')) 
      model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

      model.add(Conv2D(128,kernel_size=(3,3),padding='same')) 
      model.add(Conv2D(128,kernel_size=(3,3),padding='same')) 
      model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

      model.add(Conv2D(256,kernel_size=(3,3),padding='same')) 
      model.add(Conv2D(256,kernel_size=(3,3),padding='same')) 
      model.add(Conv2D(256,kernel_size=(3,3),padding='same')) 
      model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

      model.add(Conv2D(512,kernel_size=(3,3),padding='same')) 
      model.add(Conv2D(512,kernel_size=(3,3),padding='same')) 
      model.add(Conv2D(512,kernel_size=(3,3),padding='same')) 
      model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

      model.add(Conv2D(512,kernel_size=(3,3),padding='same')) 
      model.add(Conv2D(512,kernel_size=(3,3),padding='same')) 
      model.add(Conv2D(512,kernel_size=(3,3),padding='same')) 
      model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))


      #Flatten Duzlestirme
      model.add(Flatten())

      model.add(Dense(4096,activation=j))  #64 noron var
      model.add(Dropout(0.5))
      model.add(Dense(4096,activation=j)) 
      model.add(Dropout(0.5))
      #model.add(Dense(64,activation=j))
      #model.add(Dense(512,activation=j))
      #model.add(Dropout(0.5))   #Dropout Factorization
      model.add(Dense(3))   #3 tane class
      model.add(Activation(m))

      #Model Derleme
      model.compile(loss='categorical_crossentropy', optimizer='adam',  metrics=['accuracy'])

      #modelin eğitimi
      history=model.fit(
          train_generator,
          steps_per_epoch= train_ornek_sayisi // batch_size,
          epochs=epochs,
          validation_data=validation_generator,
          validation_steps= validation_ornek_sayisi // batch_size,
          verbose=1)
      

      #_, test_acc = model.evaluate(validation_generator, verbose=1)
      print("Train Acc : %.2f%% " % (history.history['accuracy'][-1]*100))
      print("Test Acc : %.2f%% " % (history.history['val_accuracy'][-1] * 100))
      print("\n")
      model.save("vgg16_"+str(i)+"_"+str(sayac)+"_"+str(round(history.history['val_accuracy'][-1] * 100.0, 2))+".h5")


      sayac += 1







Conv Function: relu, Traning Function: relu, Transfer Function: softmax, Epochs: 98


NameError: ignored